In [121]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [1]:
import sherpa
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


import time

import sys
sys.path.append('/home/yusukemh/github/yusukemh/StatisticalDownscaling/writeup')
from config import C_COMMON, C_GRID, C_SINGLE, FILENAME
from util import load_data, NeuralNetwork

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler


import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression as LR
from tqdm import tqdm

# enable autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import L1, L2

def define_model(
    input_dim=20,
    n_units=512,
    activation='selu',#selu
    learning_rate=0.00001,
    loss='mse',
    batch_size=64
):
    inputs = Input(shape=(input_dim))
    x = Dense(units=n_units, activation=activation, kernel_regularizer=L2(l2=0.01))(inputs)
    x = Dropout(rate=0.5)(x)
    x = Dense(units=n_units, activation=activation, kernel_regularizer=L2(l2=0.01))(x)
    x = Dropout(rate=0.5)(x)
    x = Dense(units=n_units, activation=activation, kernel_regularizer=L2(l2=0.01))(x)
    x = Dropout(rate=0.5)(x)# serves as regularization
    outputs = Dense(units=1, activation='softplus')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
        loss=loss,
        metrics=[RootMeanSquaredError()]
    )
    return model, batch_size


In [3]:
columns = C_SINGLE
n_run = 100

df_train, df_test = load_data(columns + C_COMMON, FILENAME)

In [16]:
n_model = NeuralNetwork(
    params={
        'n_units': 815,
        'learning_rate': 0.00012633633813255,
        'input_dim': 16,
        'batch_size': 64,
        'loss': 'mse'
    },
    columns=columns,
    model_func=define_model
)

In [19]:
n_model.evaluate_by_station(df_train, df_test, skn=54)

Epoch 1/1000
8/8 [==============================] - 1s 27ms/step - loss: 17.9257 - root_mean_squared_error: 1.1636 - val_loss: 17.0378 - val_root_mean_squared_error: 0.7728
Epoch 2/1000
8/8 [==============================] - 0s 8ms/step - loss: 17.1639 - root_mean_squared_error: 0.8731 - val_loss: 16.6429 - val_root_mean_squared_error: 0.6090
Epoch 3/1000
8/8 [==============================] - 0s 7ms/step - loss: 16.9878 - root_mean_squared_error: 0.8707 - val_loss: 16.4245 - val_root_mean_squared_error: 0.5745
Epoch 4/1000
8/8 [==============================] - 0s 7ms/step - loss: 16.6158 - root_mean_squared_error: 0.7513 - val_loss: 16.1642 - val_root_mean_squared_error: 0.5021
Epoch 5/1000
8/8 [==============================] - 0s 7ms/step - loss: 16.4736 - root_mean_squared_error: 0.7782 - val_loss: 15.9757 - val_root_mean_squared_error: 0.4997
Epoch 6/1000
8/8 [==============================] - 0s 7ms/step - loss: 16.3365 - root_mean_squared_error: 0.8096 - val_loss: 15.7606 - val

{'skn': 54, 'rmse_nn': 4.746579814446253, 'mae_nn': 3.1079463704427073}

In [34]:
# set up sherpa
parameters = [
    sherpa.Discrete('n_units', [256, 1024]),
    sherpa.Continuous('learning_rate', [0.00001, 0.01]),
    sherpa.Choice('batch_size', [64, 128, 192, 256, 512]),
    sherpa.Choice('loss', ['mse', 'mae'])
]
study = sherpa.Study(
    parameters=parameters,
    algorithm=sherpa.algorithms.RandomSearch(max_num_trials=n_run),
    lower_is_better=True
)

dfs = []
for i, trial in enumerate(study):
    # obtain hyperparameters
    params = {key: val for key, val in trial.parameters.items()}
    params['input_dim'] = len(columns)

    for skn in tqdm(df_train['skn'].unique()):
        model = NeuralNetwork(
            model_func=define_model,
            params=params,
            columns=columns
        )
        ret = model.cross_val_predict(df_train, skn)
        df = pd.DataFrame()
        df['trial_id'] = i,
        df['rmse'] = [ret['rmse']]
        df['mae'] = [ret['mae']]
        df['skn'] = [skn]
        for key, val in params.items():
            df[key] = [val]
        dfs.append(df)

INFO:sherpa.core:
-------------------------------------------------------
SHERPA Dashboard running. Access via
http://10.100.11.207:8887 if on a cluster or
http://localhost:8887 if running locally.
-------------------------------------------------------


 * Serving Flask app 'sherpa.app.app' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


100%|██████████| 24/24 [00:00<00:00, 338.10it/s]


# get the result

In [53]:
df_report = pd.read_csv('nn_report_500_single.csv')

In [54]:
df_groupby = df_report.groupby(by='trial_id').mean() 

In [55]:
df_groupby

,Unnamed: 0,rmse,mae,epochs,skn,n_units,learning_rate,batch_size,input_dim
trial_id,,,,,,,,,
0,0.0,9.704507,7.457255,27.875000,564.620833,688.0,0.002495,512.0,16.0
1,0.0,9.978413,7.537671,25.541667,564.620833,555.0,0.004155,256.0,16.0
2,0.0,9.877413,7.234314,42.166667,564.620833,394.0,0.003582,192.0,16.0
3,0.0,10.318749,7.989835,22.750000,564.620833,441.0,0.008499,512.0,16.0
4,0.0,12.335918,9.390875,21.375000,564.620833,525.0,0.004710,64.0,16.0
...,...,...,...,...,...,...,...,...,...
495,0.0,12.664232,9.693049,25.500000,564.620833,600.0,0.002289,64.0,16.0
496,0.0,10.341678,8.116387,23.291667,564.620833,421.0,0.007488,512.0,16.0
497,0.0,10.002604,7.662700,27.291667,564.620833,945.0,0.001313,512.0,16.0


In [60]:
objective = 'rmse'
idx_best_objective = df_groupby[objective].argmin()
df_report.query(f'trial_id == {idx_best_objective}').iloc[0][['n_units', 'learning_rate', 'input_dim', 'batch_size', 'loss']].to_dict()
# df_groupby.iloc[idx_best_objective]# [['n_units', 'learning_rate', 'input_dim', 'batch_size', 'loss']].to_dict()

{'n_units': 835,
 'learning_rate': 0.0002633633813255,
 'input_dim': 16,
 'batch_size': 64,
 'loss': 'mse'}

In [32]:
objective = 'rmse'
params = df_groupby[df_groupby[objective] == df_groupby[objective].min()][['n_units', 'learning_rate', 'batch_size', 'input_dim']].iloc[0].to_dict()
{key: val if key == 'learning_rate' else int(val) for key, val in params.items()}

{'n_units': 835,
 'learning_rate': 0.0002633633813255,
 'batch_size': 64,
 'input_dim': 16}

In [481]:
params = {}
idx = min_mse_index
params['input_dim'] = int(df_report[df_report['trial_id'] == idx].iloc[0]['input_dim'])
params['n_units'] = int(df_report[df_report['trial_id'] == idx].iloc[0]['n_units'])
params['learning_rate'] = float(df_report[df_report['trial_id'] == idx].iloc[0]['learning_rate'])
params['loss'] = df_report[df_report['trial_id'] == idx].iloc[0]['loss']
batch_size = df_report[df_report['trial_id'] == idx].iloc[0]['batch_size']

In [483]:
df_report[df_report['trial_id'] == idx]

,Unnamed: 0,input_dim,n_units,learning_rate,loss,skn,batch_size,mae,rmse,trial_id
1776,0,16,362.231815,0.000695,mse,54.0,128,3.433046,5.296056,74
1777,0,16,362.231815,0.000695,mse,79.0,128,3.886207,5.973462,74
1778,0,16,362.231815,0.000695,mse,338.0,128,2.544070,4.762007,74
1779,0,16,362.231815,0.000695,mse,250.0,128,1.394395,2.049286,74
1780,0,16,362.231815,0.000695,mse,267.0,128,1.505138,2.224552,74
1781,0,16,362.231815,0.000695,mse,296.1,128,0.799617,1.681203,74
1782,0,16,362.231815,0.000695,mse,311.0,128,0.796940,1.520546,74
1783,0,16,362.231815,0.000695,mse,396.0,128,1.051577,1.784216,74
1784,0,16,362.231815,0.000695,mse,400.0,128,1.138457,1.805463,74
1785,0,16,362.231815,0.000695,mse,406.0,128,1.241432,1.891523,74


In [490]:
list_ypred = []
list_ytrue = []
for fold in range(5):
    x_train, x_test, y_train, y_test= prepare_dataset(df_inner_split, skn=skn, inner_fold=0)
    linear_regression = LinearRegression()
    linear_regression.fit(x_train, y_train)
    yhat = linear_regression.predict(x_test)
    list_ytrue.append(y_test)
    list_ypred.append(yhat)
mean_absolute_error(list_ytrue, list_ypred)

4.524202419975154